## Fusión de Inventarios Semanales de la **EIA** con Fechas Correspondientes Scrapeadas de **Investing**

In [3]:
import pandas as pd

C:\Users\Usuario\AppData\Local\Temp\ipykernel_34068\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Cargar datos de producción, importación, exportación e inventario del crudo en EE.UU.
df_produccion = pd.read_csv("Weekly_U.S._Field_Production_of_Crude_Oil.csv", header=None, names=['Date', 'produccion_diaria_promedio'])
df_importacion = pd.read_csv("Weekly_U.S._Imports_of_Crude_Oil.csv", header=None, names=['Date_1', 'importacion_diaria_promedio'])
df_exportacion = pd.read_csv("Weekly_U.S._Exports_of_Crude_Oil.csv", header=None, names=['Date_2', 'exportacion_diaria_promedio'])
df_inventario = pd.read_csv("Weekly_U.S._Ending_Stocks_excluding_SPR_of_Crude_Oil.csv", header=None, names=['Date_3', 'inventario_final_semanal'])

In [5]:
#Reset index
df_produccion = df_produccion.iloc[5:,:].reset_index(drop=True)
df_importacion = df_importacion.iloc[5:,:].reset_index(drop=True)
df_exportacion = df_exportacion.iloc[5:,:].reset_index(drop=True)
df_inventario = df_inventario.iloc[5:,:].reset_index(drop=True)

In [8]:
#Concatenamos los cuatro datasets.
df_cl_macro = pd.concat([df_produccion, df_importacion, df_exportacion, df_inventario], axis=1)

In [9]:
#Borramos fechas repetidas
df_cl_macro.drop(columns=["Date_1","Date_2", "Date_3"],inplace=True)

#Eliminamos datos NaN para tener solo datos completos
df_cl_macro.dropna()

In [10]:
# Rellenar los valores faltantes en la columna 'inventario_final_semanal' usando el último valor conocido (forward fill)
df_inventario['inventario_final_semanal'] = df_inventario['inventario_final_semanal'].ffill()

In [12]:
# Cargar y procesar fechas de publicación
fechas = pd.read_csv("inventario-CL.csv")
fechas.rename(columns={"Fecha de publicación": "Date"}, inplace=True)
fechas['Date'] = pd.to_datetime(fechas['Date'], dayfirst=True)

# Cargar y procesar inventarios
inventario = pd.read_csv("Inventarios_null.csv")
inventario['Date'] = pd.to_datetime(inventario['Date']).reset_index(drop=True)
inventario.drop(columns="Unnamed: 0")

,Date,Hora,Actual,Previsión,Anterior
0,2024-09-11,16:30,NaN,NaN,"-6,873M"
1,2024-09-05,17:00,"-6,873M","-0,600M","-0,846M"
2,2024-08-28,16:30,"-0,846M","-2,700M","-4,649M"
3,2024-08-21,16:30,"-4,649M","-2,000M","1,357M"
4,2024-08-14,16:30,"1,357M","-1,900M","-3,728M"
...,...,...,...,...,...
745,2010-06-03,17:00,"-1,900M",NaN,"2,400M"
746,2010-05-26,16:30,"2,400M",NaN,"0,200M"
747,2010-05-19,16:30,"0,200M",NaN,"1,900M"
748,2010-05-12,16:30,"1,900M",NaN,"2,800M"


In [13]:
# Cargar y procesar el archivo de inventarios
inventario = pd.read_csv("Inventarios_null.csv")
inventario['Date'] = pd.to_datetime(inventario['Date']).reset_index(drop=True)
inventario.drop(columns="Unnamed: 0")

,Date,inventario_final_semanal
0,2014-01-02,NaN
1,2014-01-03,326737.0
2,2014-01-06,326737.0
3,2014-01-07,326737.0
4,2014-01-08,326737.0
...,...,...
2657,2024-07-25,436485.0
2658,2024-07-26,433049.0
2659,2024-07-29,433049.0
2660,2024-07-30,433049.0


In [14]:
# Realizar el left join basado en la columna Date
merged_df = fechas.merge(inventario, how='inner', on='Date')

inventario_final = merged_df[["Date","inventario_final_semanal"]]
inventario_final